### Imports

In [1]:
import requests
import time
import json
from datetime import datetime

### Consants

In [2]:
API_KEY = "GyGyGeRAriMKyywnoAJNkT2eFnS2xuQlxDcj6LGy"
season = "2025"
season_type = "REG"
language = "en"
access_level = "trial"
format = "json"

### Game Ids

In [4]:

def get_game_ids(api_key, season=season, season_type=season_type):
    url = f"https://api.sportradar.com/nfl/official/trial/v7/en/games/{season}/{season_type}/schedule.json"
    headers = {"x-api-key": api_key}
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except Exception as e:
        print(f"Error fetching schedule: {e}")
        return []
    
    data = response.json()
    game_ids = []

    for week in data.get("weeks", []):
        for game in week.get("games", []):
            if game.get("status") == "closed":
                game_ids.append(game["id"])

    print(f"Found {len(game_ids)} closed games.")
    return game_ids

# Test it
closed_game_ids = get_game_ids(API_KEY)
print(closed_game_ids[:5])  # print first 5 game IDs to check


Error fetching schedule: 429 Client Error: Too Many Requests for url: https://api.sportradar.com/nfl/official/trial/v7/en/games/2025/REG/schedule.json
[]


### Teams in each game

In [5]:
def get_game_teams_by_ids(api_key, game_ids_list, season=season, season_type=season_type):
    url = f"https://api.sportradar.com/nfl/official/trial/v7/en/games/{season}/{season_type}/schedule.json"
    headers = {"x-api-key": api_key}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch schedule: {response.status_code}")
        return []

    data = response.json()
    games_info = []

    for week in data.get("weeks", []):
        for game in week.get("games", []):
            if game.get("id") in game_ids_list:
                # Check keys exist before accessing
                if 'home' in game and 'away' in game:
                    games_info.append({
                        "game_id": game["id"],
                        "home": game["home"].get("alias", "unknown"),
                        "away": game["away"].get("alias", "unknown")
                    })
                else:
                    print(f"Missing 'home' or 'away' in game: {game.get('id', 'unknown')}")
    return games_info

get_game_teams_by_ids(API_KEY, get_game_ids(API_KEY, season=season, season_type=season_type), season=season, season_type=season_type)[:5]


Error fetching schedule: 429 Client Error: Too Many Requests for url: https://api.sportradar.com/nfl/official/trial/v7/en/games/2025/REG/schedule.json
Failed to fetch schedule: 429


[]

### Records

In [5]:
def get_team_records(api_key, season, season_type):
    url = (
        f"https://api.sportradar.com/nfl/official/{access_level}/v7/{language}/"
        f"seasons/{season}/{season_type}/standings/season.{format}?api_key={api_key}"
    )

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error fetching standings: {response.status_code}")
        return {}

    data = response.json()
    records = {}

    # Loop through conferences, divisions, and teams
    for conference in data.get("conferences", []):
        for division in conference.get("divisions", []):
            for team_record in division.get("teams", []):
                try:
                    # Extract team info
                    team_alias = team_record.get("alias", "UNK")
                    wins = team_record.get("wins", 0)
                    losses = team_record.get("losses", 0)

                    # Skip if any of these values are missing
                    if team_alias == "UNK":
                        print("Skipping a team without 'team' info.")
                        continue
                    
                    # Store records as net wins (wins - losses)
                    records[team_alias] = wins - losses
                except KeyError as e:
                    print(f"Skipping a team with missing data: {team_record}, Error: {e}")

    return records

# Run it
records = get_team_records(API_KEY, season, season_type)
print(records)


{'NYG': -3, 'WAS': -1, 'PHI': 3, 'DAL': 0, 'NO': -5, 'TB': 3, 'ATL': 0, 'CAR': 1, 'LA': 3, 'SEA': 3, 'ARI': -3, 'SF': 3, 'MIN': 0, 'CHI': 2, 'GB': 3, 'DET': 3, 'LAC': 1, 'KC': 1, 'LV': -3, 'DEN': 3, 'MIA': -5, 'NYJ': -7, 'BUF': 2, 'NE': 3, 'IND': 5, 'HOU': -2, 'TEN': -5, 'JAC': 1, 'CIN': -1, 'PIT': 2, 'CLE': -3, 'BAL': -4}


### Active Roster

In [9]:
def get_active_in_game_rosters(api_key, game_ids, sleep_seconds=1):
    """
    Fetches the game‐roster for each given game ID, and returns only players
    whose `in_game_status` is "active" for each team (home & away).

    Args:
        api_key (str): Sportradar API key.
        game_ids (list of str): List of game‑IDs to fetch rosters for.
        sleep_seconds (float): Time to wait between requests to respect rate limits.
    
    Returns:
        dict: game_id → {
                    "home_team": alias,
                    "away_team": alias,
                    "home_active_roster": [ {player data} ],
                    "away_active_roster": [ {player data} ]
                }
    """
    base_url = "https://api.sportradar.com/nfl/official/trial/v7/en/games/{}/roster.json"
    headers = {"x-api-key": api_key}
    result = {}

    for game_id in game_ids:
        url = base_url.format(game_id)
        try:
            resp = requests.get(url, headers=headers)
            resp.raise_for_status()
            data = resp.json()
        except requests.exceptions.RequestException as e:
            print(f"❌ Error fetching roster for game {game_id}: {e}")
            continue

        home = data.get("home", {})
        away = data.get("away", {})
        home_alias = home.get("alias", "UNKNOWN")
        away_alias = away.get("alias", "UNKNOWN")

        def filter_active(players):
            return [
                {
                    "id": p.get("id"),
                    "name": p.get("name"),
                    "position": p.get("position"),
                    "jersey": p.get("jersey_number"),
                    "status": p.get("status"),
                    "in_game_status": p.get("in_game_status"),
                }
                for p in (players or [])
                if p.get("in_game_status") == "active"
            ]

        home_active = filter_active(home.get("players", []))
        away_active = filter_active(away.get("players", []))

        result[game_id] = {
            "home_team": home_alias,
            "away_team": away_alias,
            "home_active_roster": home_active,
            "away_active_roster": away_active
        }

        print(f"✅ Processed game {game_id}: {len(home_active)} home / {len(away_active)} away active players")
        time.sleep(3)
        

    print("✅ Saved active in-game rosters to in_game_rosters.json")
    return result







In [10]:
# Example: Pull rosters for 3 closed games
closed_game_ids = get_game_ids(API_KEY)[:3]

in_game_rosters = get_active_in_game_rosters(API_KEY, closed_game_ids[:3])

# Print results for each game
for game_id, info in in_game_rosters.items():
    print(f"\nGame ID: {game_id}")
    print(f"{info['home_team']} Active Players: {len(info['home_active_roster'])}")
    print(f"{info['away_team']} Active Players: {len(info['away_active_roster'])}")

    print("Sample Home Player:", info['home_active_roster'][0] if info['home_active_roster'] else "None")
    print("Sample Away Player:", info['away_active_roster'][0] if info['away_active_roster'] else "None")


Found 108 closed games.
✅ Processed game 56779053-89da-4939-bc22-9669ae1fe05a: 72 home / 79 away active players
✅ Processed game f895a349-36ec-4b9a-b7a3-7830ea95f07a: 78 home / 78 away active players
✅ Processed game 0ab5e37d-0f79-40c6-b536-ff88c7be5286: 77 home / 72 away active players
✅ Saved active in-game rosters to in_game_rosters.json

Game ID: 56779053-89da-4939-bc22-9669ae1fe05a
PHI Active Players: 72
DAL Active Players: 79
Sample Home Player: {'id': 'f1a6c070-572e-4728-b5ab-7765bb31f9e2', 'name': 'Tyler Steen', 'position': 'OL', 'jersey': None, 'status': 'started', 'in_game_status': 'active'}
Sample Away Player: {'id': 'a72ea15b-5199-4101-a300-846e1c655add', 'name': 'CeeDee Lamb', 'position': 'WR', 'jersey': None, 'status': 'started', 'in_game_status': 'active'}

Game ID: f895a349-36ec-4b9a-b7a3-7830ea95f07a
LAC Active Players: 78
KC Active Players: 78
Sample Home Player: {'id': 'f475c020-23d1-11f0-a1b4-0388130a8f25', 'name': 'Garmon Randolph', 'position': 'OLB', 'jersey': Non

### Datetime

In [11]:
import requests
from datetime import datetime
import time

def get_game_datetimes(api_key, season="2025", season_type="REG"):
    """
    Retrieves the date and time (as datetime objects) of all games for the given NFL season.
    
    Args:
        api_key (str): Sportradar API key.
        season (str): Season year, e.g., "2025".
        season_type (str): Season type, e.g., "REG", "PRE", or "POST".
    
    Returns:
        dict: Mapping of game_id → {
            'home': team_alias,
            'away': team_alias,
            'scheduled': datetime object (UTC)
        }
    """
    url = f"https://api.sportradar.com/nfl/official/trial/v7/en/games/{season}/{season_type}/schedule.json"
    headers = {"x-api-key": api_key}
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"❌ Error fetching schedule: {e}")
        return {}

    data = response.json()
    games_by_id = {}

    for week in data.get("weeks", []):
        for game in week.get("games", []):
            game_id = game.get("id")
            scheduled_str = game.get("scheduled")  # ISO 8601 UTC time string
            if not scheduled_str:
                continue

            try:
                scheduled_dt = datetime.fromisoformat(scheduled_str.replace("Z", "+00:00"))  # Parse to aware datetime
            except Exception as e:
                print(f"⚠️ Could not parse date for game {game_id}: {e}")
                continue

            games_by_id[game_id] = {
                "home": game.get("home", {}).get("alias", "UNK"),
                "away": game.get("away", {}).get("alias", "UNK"),
                "scheduled": scheduled_dt,
            }

    print(f"✅ Retrieved {len(games_by_id)} games with date/time info.")
    return games_by_id


In [12]:
game_datetimes = get_game_datetimes(API_KEY)

# Print a few results
for gid, info in list(game_datetimes.items())[:5]:
    print(f"Game ID: {gid}")
    print(f"{info['away']} @ {info['home']} on {info['scheduled'].strftime('%Y-%m-%d %H:%M:%S %Z')}")


✅ Retrieved 272 games with date/time info.
Game ID: 56779053-89da-4939-bc22-9669ae1fe05a
DAL @ PHI on 2025-09-05 00:20:00 UTC
Game ID: f895a349-36ec-4b9a-b7a3-7830ea95f07a
KC @ LAC on 2025-09-06 00:00:00 UTC
Game ID: 0ab5e37d-0f79-40c6-b536-ff88c7be5286
TB @ ATL on 2025-09-07 17:00:00 UTC
Game ID: 56ae5686-bcc1-41a5-8988-66c3c7741a05
PIT @ NYJ on 2025-09-07 17:00:00 UTC
Game ID: 5894a475-84a9-4aa1-b505-e72281da8b1d
MIA @ IND on 2025-09-07 17:00:00 UTC


### Rush Yards

In [14]:
import requests
import time
import random

def get_schedule_with_key_in_url(api_key, season="2025", season_type="REG"):
    """
    Fetch the NFL schedule from Sportradar API.
    Args:
        api_key (str): API key for accessing the Sportradar data.
        season (str): Season year, default '2025'.
        season_type (str): Season type, default 'REG'.
    Returns:
        dict: JSON data with the NFL schedule.
    """
    url_schedule = f"https://api.sportradar.com/nfl/official/trial/v7/en/games/{season}/{season_type}/schedule.json"
    headers = {"x-api-key": api_key}
    try:
        response = requests.get(url_schedule, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as err:
        if err.response.status_code == 403:
            print(f"Access forbidden. Response: {err.response.text}")
        else:
            print(f"Error fetching schedule: {err}")
        return None

def get_game_boxscore(api_key, game_id):
    """
    Fetch the boxscore for a specific game.
    Args:
        api_key (str): API key for accessing the Sportradar data.
        game_id (str): Unique game identifier.
    Returns:
        dict: Boxscore data for the game, or 'NA' if not available.
    """
    url_boxscore = f"https://api.sportradar.com/nfl/official/trial/v7/en/games/{game_id}/boxscore.json"
    headers = {"x-api-key": api_key}
    
    try:
        response = requests.get(url_boxscore, headers=headers)
        response.raise_for_status()
        boxscore_data = response.json()

        # Check if we have a valid game response
        if "game" not in boxscore_data:
            print(f"Game {game_id}: No game data found.")
            return {"home_avg_rushing_yards": "NA", "away_avg_rushing_yards": "NA"}

        # Debug: log the entire response for inspection
        print(f"Boxscore Data for {game_id}: {boxscore_data}")

        # Extract rushing yards for the home and away teams
        teams_stats = boxscore_data.get("game", {}).get("teams", [])
        rushing_stats = {}

        for team_stat in teams_stats:
            alias = team_stat.get("alias")
            rushing_yards = team_stat.get("statistics", {}).get("rushing_yards", "NA")
            rushing_stats[alias] = rushing_yards

        # Check if rushing yards are missing in the response and log
        if not rushing_stats:
            print(f"Game {game_id}: Rushing stats missing for home and away teams.")
        
        return rushing_stats

    except requests.exceptions.HTTPError as err:
        if err.response.status_code == 404:
            print(f"Game {game_id} not found, skipping.")
        elif err.response.status_code == 429:
            print("Too many requests. Sleeping for 3 seconds...")
            time.sleep(3)
        else:
            print(f"Failed to fetch stats for game {game_id}: {err}")
        return {"home_avg_rushing_yards": "NA", "away_avg_rushing_yards": "NA"}

def process_games(api_key, season="2025", season_type="REG"):
    """
    Process and fetch rushing yards for random games.
    Args:
        api_key (str): API key for accessing the Sportradar data.
        season (str): Season year, default '2025'.
        season_type (str): Season type, default 'REG'.
    Returns:
        list: A list of processed game stats with rushing yards data.
    """
    schedule_data = get_schedule_with_key_in_url(api_key, season, season_type)
    
    if not schedule_data:
        print("Failed to retrieve schedule data.")
        return []
    
    processed_games = []

    # Process a random selection of 5 games
    game_ids = [game.get("id") for week in schedule_data.get("weeks", []) for game in week.get("games", [])]
    random_game_ids = random.sample(game_ids, 5)

    for game_id in random_game_ids:
        home_team = next((game.get("home", {}).get("alias") for week in schedule_data.get("weeks", []) for game in week.get("games", []) if game.get("id") == game_id), None)
        away_team = next((game.get("away", {}).get("alias") for week in schedule_data.get("weeks", []) for game in week.get("games", []) if game.get("id") == game_id), None)

        if home_team and away_team:
            rushing_stats = get_game_boxscore(api_key, game_id)

            processed_games.append({
                "game_id": game_id,
                "home_team": home_team,
                "away_team": away_team,
                "home_avg_rushing_yards": rushing_stats.get(home_team, "NA"),
                "away_avg_rushing_yards": rushing_stats.get(away_team, "NA")
            })
        
        # To avoid hitting rate limits, add a short sleep between requests
        time.sleep(0.5)

    return processed_games




In [15]:
games = process_games(API_KEY)

for game in games:
    print(game)

Game 4f446eeb-1c14-436b-9321-e1c7b6276da5: No game data found.
Game cc18ff20-f2e2-4d07-9ed1-40320efb9e7b: No game data found.
Too many requests. Sleeping for 3 seconds...
Game b629b0e9-5448-4223-8dd2-e924aabe0183: No game data found.
Game 8ea39b7e-0170-41fa-88db-36179a59c735: No game data found.
{'game_id': '4f446eeb-1c14-436b-9321-e1c7b6276da5', 'home_team': 'SF', 'away_team': 'JAC', 'home_avg_rushing_yards': 'NA', 'away_avg_rushing_yards': 'NA'}
{'game_id': 'cc18ff20-f2e2-4d07-9ed1-40320efb9e7b', 'home_team': 'IND', 'away_team': 'SF', 'home_avg_rushing_yards': 'NA', 'away_avg_rushing_yards': 'NA'}
{'game_id': '926f7b80-6936-4b74-bccf-a98e914df0d5', 'home_team': 'NYJ', 'away_team': 'CAR', 'home_avg_rushing_yards': 'NA', 'away_avg_rushing_yards': 'NA'}
{'game_id': 'b629b0e9-5448-4223-8dd2-e924aabe0183', 'home_team': 'SEA', 'away_team': 'HOU', 'home_avg_rushing_yards': 'NA', 'away_avg_rushing_yards': 'NA'}
{'game_id': '8ea39b7e-0170-41fa-88db-36179a59c735', 'home_team': 'DEN', 'away_tea

In [63]:
game_stats

[{'game_id': '54d3eead-8407-44ca-a545-53754b4c00e8',
  'home_avg_rushing_yards': 'NA',
  'away_avg_rushing_yards': 'NA'},
 {'game_id': 'c504c9b7-1b1b-487f-8d28-160fb6b34c54',
  'home_avg_rushing_yards': 'NA',
  'away_avg_rushing_yards': 'NA'},
 {'game_id': '4c314a4e-d99b-42ef-9ecb-52ad746d4dfa',
  'home_avg_rushing_yards': 'NA',
  'away_avg_rushing_yards': 'NA'},
 {'game_id': 'f4d94f7a-8a51-4b18-b58c-11008a76f1fb',
  'home_avg_rushing_yards': 'NA',
  'away_avg_rushing_yards': 'NA'},
 {'game_id': 'e9d69acd-909d-4b77-8f53-16ffc8885d6a',
  'home_avg_rushing_yards': 'NA',
  'away_avg_rushing_yards': 'NA'}]

In [32]:
import requests
import time

API_KEY = "GyGyGeRAriMKyywnoAJNkT2eFnS2xuQlxDcj6LGy"
access_level = "trial"
language = "en"
game_id = "c504c9b7-1b1b-487f-8d28-160fb6b34c54"  # Example game ID, replace with the actual game ID

def get_rushing_yards(api_key, game_id, retries=5, delay=1):
    url = f"https://api.sportradar.com/nfl/official/{access_level}/v7/{language}/games/{game_id}/boxscore.json"
    headers = {"x-api-key": api_key}

    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise an exception if status code is not 200
            
            # Parse the JSON response
            data = response.json()

            # Extract rushing yards for both teams
            home_team_rushing = 0
            away_team_rushing = 0

            if 'home' in data['game'] and 'away' in data['game']:
                home_team_rushing = data['game']['home']['statistics'].get('rushing', {}).get('yards', 0)
                away_team_rushing = data['game']['away']['statistics'].get('rushing', {}).get('yards', 0)

            return home_team_rushing, away_team_rushing

        except requests.exceptions.RequestException as e:
            if response.status_code == 429:  # Rate limit error
                print(f"Rate limit hit. Retrying in {delay} seconds...")
                time.sleep(delay)  # Wait before retrying
                delay *= 2  # Exponential backoff
            else:
                print(f"Error fetching boxscore: {e}")
                return None

    print("Exceeded maximum retries.")
    return None

# Example usage:
rushing_yards = get_rushing_yards(API_KEY, game_id)

if rushing_yards is None:
    print("Could not retrieve rushing yards.")
else:
    home_rushing, away_rushing = rushing_yards
    print(f"Home Team Rushing Yards: {home_rushing}")
    print(f"Away Team Rushing Yards: {away_rushing}")


Rate limit hit. Retrying in 1 seconds...
Rate limit hit. Retrying in 2 seconds...
Rate limit hit. Retrying in 4 seconds...
Rate limit hit. Retrying in 8 seconds...
Rate limit hit. Retrying in 16 seconds...
Exceeded maximum retries.
Could not retrieve rushing yards.
